# Stitching together product solutions on regions

Can we do this?

In [ ]:
import ngsolve as ng
import numpy as np

from pyeigfeast.spectralproj.ngs import SpectralProjNG, NGvecs
from ngsolve.webgui import Draw
from netgen.geom2d import CSG2d, Rectangle
from numpy import pi

In [ ]:

geo = CSG2d()

# rectA = Rectangle(pmin=(-pi,0), pmax=(0,pi), mat="mat3", bc="outer")
rectB = Rectangle(pmin=(-pi,-pi), pmax=(0,0), mat="left", bc='outer', right='center')
rectC = Rectangle(pmin=(0,-pi), pmax=(pi,0), mat="right", bc='outer', left='center')

# geo.Add(rectA)
geo.Add(rectB)
geo.Add(rectC)

mesh = ng.Mesh(geo.GenerateMesh(maxh=0.3))

Draw(mesh)

# PDE to solve is $-\Delta u + V u = Z^2 u$

The eigenvalue is Z, V is our potential function which we define below

In [ ]:
n_core = 1.48
n_clad = 1.45
wavelength = 1.8e-6
L = .2e-5
k0 = 2 * pi / wavelength

N = ng.CoefficientFunction([n_core, n_clad])

n0 = n_clad
V = (k0*L)**2 * (n0**2-N**2)

In [ ]:
Draw(V,mesh)

In [ ]:
fes = ng.H1(mesh, order=3, complex=True, dirichlet='outer')

u, v = fes.TnT()

a = ng.BilinearForm(fes)
a += (ng.grad(u) * ng.grad(v) + V * u * v) * ng.dx

b = ng.BilinearForm(fes)
b += u * v * ng.dx

with ng.TaskManager():
    a.Assemble()
    b.Assemble()

In [ ]:
beta_l, beta_r = k0 * n_clad, k0 * n_core
betas = np.array([beta_l, beta_r])
right, left = (L * k0 * n0)**2 - (L * betas)**2

In [ ]:
left, right

In [ ]:
nspan=6

ctr = (right+left)/2
rad = (right-left)/2

P = SpectralProjNG(fes, a.mat, b.mat, radius=rad, center=ctr, npts=6)

Y = NGvecs(fes, nspan, M=b.mat)
Y.setrandom(seed=1)

Zsqrs, Y, history, _ = P.feast(Y, hermitian=True, niterations=100, nrestarts=0, stop_tol=1e-9)


In [ ]:
for i, y in enumerate(Y):
    if i <5:
        Draw(1e1*y, mesh)
    else:
        pass

# Can we get these solutions by stitching together product solutions on each of the two regions?

We know that we can solve this universally using separation of variables, but can we do it on each region and then stitch them together to get an (equivalent) eigenvalue equation?


# Are these product solutions?

We've been checking on this for other things, worth seeing it here where we (nearly) know these are product solutions.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
Sx = np.linspace(0, pi, 301)
Sy = np.linspace(-pi, 0, 301)

In [ ]:
x0, y0 = pi/2,-pi/2
f = Y[0]
xs, ys = [mesh(sx, y0) for sx in Sx], [mesh(x0, sy) for sy in Sy]
fxs = np.array([f(xpt) for xpt in xs])
fys = np.array([f(ypt) for ypt in ys])

In [ ]:
%matplotlib inline
plt.plot(S, fxs.real)
# plt.plot(S, fxs.imag)



In [ ]:
plt.plot(S, fys.real)
# plt.plot(S, fys.imag)

# Create product function

To see if the above really are product functions, we should be able to recreate them using the two cross sections we have

In [ ]:
from mpl_toolkits import mplot3d

In [ ]:
u_0 = f(mesh(x0,y0))

In [ ]:
u_0

In [ ]:
Xs, Ys = np.meshgrid(S,S)
Z = np.outer(fxs, fys) / u_0

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection='3d')
ax.plot_surface(Xs, Ys, Z.real, cmap='viridis');
# ax.plot_surface(Xs, Ys, Z.imag, cmap='viridis')

In [ ]:
Z_true = np.zeros_like(Xs, dtype=complex)

In [ ]:
for i,x in enumerate(Xs):
    for j, y in enumerate(Ys):
        Z_true[i,j] = f(mesh(x[i],y[j]))

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(Xs, Ys, Z_true.imag, cmap='viridis')
ax.plot_surface(Xs, Ys, Z.imag, cmap='turbo')

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(Xs, Ys, (Z/Z_true).real, cmap='viridis')

## The solution on the left region is unequivocally a product function.

We can probably back out the parameters for the sin functions that form the product.  But underlying question is how can we find this solution by stitching together product solutions on each region?  Let's also do this for the L shape and see if these are also product solutions

In [ ]:

geo = CSG2d()

rectA = Rectangle(pmin=(-pi,0), pmax=(0,pi), mat="top", bc="outer", bottom='center')
rectB = Rectangle(pmin=(-pi,-pi), pmax=(0,0), mat="left", bc='outer', right='center', top='center')
rectC = Rectangle(pmin=(0,-pi), pmax=(pi,0), mat="right", bc='outer', left='center')

geo.Add(rectA)
geo.Add(rectB)
geo.Add(rectC)

mesh = ng.Mesh(geo.GenerateMesh(maxh=0.3))

Draw(mesh)

In [ ]:
n_core = 1.48
n_clad = 1.45
wavelength = 1.8e-6
L = .2e-5
k0 = 2 * pi / wavelength

N = ng.CoefficientFunction([n_clad, n_core, n_clad])

n0 = n_clad
V = (k0*L)**2 * (n0**2-N**2)

In [ ]:
Draw(V,mesh)

In [ ]:
fes = ng.H1(mesh, order=2, complex=True, dirichlet='outer')

u, v = fes.TnT()

a = ng.BilinearForm(fes)
a += (ng.grad(u) * ng.grad(v) + V * u * v) * ng.dx

b = ng.BilinearForm(fes)
b += u * v * ng.dx

with ng.TaskManager():
    a.Assemble()
    b.Assemble()

In [ ]:
beta_l, beta_r = k0 * n_clad, k0 * n_core
betas = np.array([beta_l, beta_r])
right, left = (L * k0 * n0)**2 - (L * betas)**2

In [ ]:
left, right

In [ ]:
nspan=15

ctr = (right+left)/2
rad = (right-left)/2

P = SpectralProjNG(fes, a.mat, b.mat, radius=rad, center=ctr, npts=8)

Y = NGvecs(fes, nspan, M=b.mat)
Y.setrandom(seed=1)

Zsqrs, Y, history, _ = P.feast(Y, hermitian=True, niterations=100, nrestarts=0, stop_tol=1e-9)


In [ ]:
for i, y in enumerate(Y):
    if i <5:
        Draw(1e1*y, mesh)
    else:
        pass

### Form the product again

In [ ]:
Sx = np.linspace(0, pi, 301)
Sy = np.linspace(-pi, 0, 301)

In [ ]:
x0, y0 = 0,-pi/2
f = Y[0]
xs, ys = [mesh(sx, y0) for sx in Sx], [mesh(x0, sy) for sy in Sy]
fxs = np.array([f(xpt) for xpt in xs])
fys = np.array([f(ypt) for ypt in ys])

In [ ]:
%matplotlib inline
plt.plot(S, fxs.real)
# plt.plot(S, fxs.imag)



In [ ]:
plt.plot(S, fys.real)
# plt.plot(S, fys.imag)

In [ ]:
u_0 = f(mesh(x0,y0))

In [ ]:
Xs, Ys = np.meshgrid(Sx,Sy)
Z = np.outer(fxs, fys) / u_0


In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection='3d')
ax.plot_surface(Xs, Ys, Z.real, cmap='viridis');


In [ ]:

Z_true = np.zeros_like(Xs, dtype=complex)

for i,x in enumerate(Xs):
    for j, y in enumerate(Ys):
        Z_true[i,j] = f(mesh(x[i],y[j]))


In [ ]:

%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(Xs, Ys, Z_true.real, cmap='viridis')
ax.plot_surface(Xs, Ys, Z.real, cmap='turbo')

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(Xs, Ys, (Z-Z_true).real, cmap='turbo')

# One more time with 4 regions

In [ ]:

geo = CSG2d()

rectA = Rectangle(pmin=(-pi,0), pmax=(0,pi), mat="top_left", bc="outer", bottom='center', right='center')
rectB = Rectangle(pmin=(0,0), pmax=(pi,pi), mat="top_right", bc="outer", bottom='center', left='center')
rectC = Rectangle(pmin=(-pi,-pi), pmax=(0,0), mat="bottom_left", bc='outer', right='center', top='center')
rectD = Rectangle(pmin=(0,-pi), pmax=(pi,0), mat="bottom_right", bc='outer', left='center', top='center')

geo.Add(rectA)
geo.Add(rectB)
geo.Add(rectC)
geo.Add(rectD)

mesh = ng.Mesh(geo.GenerateMesh(maxh=0.3))

Draw(mesh)

In [ ]:
n_core = 1.48
n_clad = 1.45
wavelength = 1.8e-6
L = .2e-5
k0 = 2 * pi / wavelength

N = ng.CoefficientFunction([n_clad, n_clad, n_core, n_clad])

n0 = n_clad
V = (k0*L)**2 * (n0**2-N**2)

In [ ]:
Draw(V,mesh)

In [ ]:
fes = ng.H1(mesh, order=2, complex=True, dirichlet='outer')

u, v = fes.TnT()

a = ng.BilinearForm(fes)
a += (ng.grad(u) * ng.grad(v) + V * u * v) * ng.dx

b = ng.BilinearForm(fes)
b += u * v * ng.dx

with ng.TaskManager():
    a.Assemble()
    b.Assemble()

In [ ]:
beta_l, beta_r = k0 * n_clad, k0 * n_core
betas = np.array([beta_l, beta_r])
right, left = (L * k0 * n0)**2 - (L * betas)**2

In [ ]:
left, right

In [ ]:
nspan=3

ctr = (right+left)/2
rad = (right-left)/2

P = SpectralProjNG(fes, a.mat, b.mat, radius=rad, center=ctr, npts=8)

Y = NGvecs(fes, nspan, M=b.mat)
Y.setrandom(seed=1)

Zsqrs, Y, history, _ = P.feast(Y, hermitian=True, niterations=100, nrestarts=0, stop_tol=1e-9)


In [ ]:
for i, y in enumerate(Y):
    if i <5:
        Draw(1e1*y, mesh)
    else:
        pass

### Form the product again

In [ ]:
Sx = np.linspace(-pi, 0, 301)
Sy = np.linspace(-pi, 0, 301)

In [ ]:
x0, y0 = -pi/2,-pi/2
f = Y[0]
xs, ys = [mesh(sx, y0) for sx in Sx], [mesh(x0, sy) for sy in Sy]
fxs = np.array([f(xpt) for xpt in xs])
fys = np.array([f(ypt) for ypt in ys])

In [ ]:
%matplotlib inline
plt.plot(Sx, fxs.real)
# plt.plot(S, fxs.imag)



In [ ]:
plt.plot(Sy, fys.real)
# plt.plot(S, fys.imag)

In [ ]:
u_0 = f(mesh(x0,y0))

In [ ]:
Xs, Ys = np.meshgrid(Sx,Sy)
Z = np.outer(fxs, fys) / u_0


In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection='3d')
ax.plot_surface(Xs, Ys, Z.real, cmap='viridis');


In [ ]:

Z_true = np.zeros_like(Xs, dtype=complex)

for i,x in enumerate(Xs):
    for j, y in enumerate(Ys):
        Z_true[i,j] = f(mesh(x[i],y[j]))


In [ ]:

%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(Xs, Ys, Z_true.imag, cmap='viridis')
ax.plot_surface(Xs, Ys, Z.imag, cmap='turbo')

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(Xs, Ys, -(Z-Z_true).real, cmap='turbo')

# Observations

In the L shape, it doesn't look like the core area is a product. Yeah, it isn't, at least by this way of checking, which seems to be totally correct.